#66
### Big Bird: Efficient Transformer for Long Sequences

[Big Bird: Transformers for Longer Sequences](https://arxiv.org/abs/2007.14062)  
[Bigbird 리뷰](https://gbdai.tistory.com/60)

- 개요
  - Big Bird는 긴 시퀀스를 효율적으로 처리하기 위해 설계된 트랜스포머 모델.
  - 트랜스포머 모델은 입력 시퀀스 길이에 따라 연산 비용이 급격히 증가하는데, Big Bird는 이를 개선하기 위해 Sparse Attention 방식을 사용.

- 핵심 아이디어
  - Self-Attention → Fully-Connected Graph
    - Self-attention은 모든 토큰 간 연결로 볼 수 있으며, 이는 fully-connected graph로 표현할 수 있.
  - Fully-Connected Graph → Sparse Random Graph
    - Self-attention graph를 sparse하게 만들면 긴 시퀀스에서도 성능을 유지하며 계산 효율성을 높일 수 있다.

- Generalized attention

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbyQMqV%2FbtqW4ijTTvx%2Fuf1b4sSx975tregXco8nL1%2Fimg.png)

- 제안된 Attention
  - 로컬(Local) Attention: 각 토큰이 근처의 일부 토큰들과만 상호 작용하여 로컬 문맥을 포착.
  - 글로벌(Global) Attention: 특정한 중요 토큰들(g)이 전체 시퀀스의 모든 토큰들과 상호작용하여 전역 정보를 포착.
  - 랜덤(Random) Attention: 각 토큰이 무작위로 선택된 토큰들과 상호작용하여 장거리 의존성을 학습.

- 참고
  - 글로벌 토큰:
    - 문서의 전체 의미를 대표하는 [CLS] 토큰을 글로벌 토큰으로 사용할 수 있. [CLS] 토큰은 주로 문서 분류나 질문-응답(QA) 작업에서 문서의 요약 정보를 제공한다.
    - 문장 쌍 기반의 작업(ex. NLI, QA)에서는 [SEP] 토큰을 사용하여 각 문장 쌍 간의 구분을 나타내고, 문맥 정보를 효과적으로 학습할 수 있다.

- 정리
  - Query가 r개의 random Key를 참고하는 attention.
  - 각 Query가 현재 위치 기준으로 양옆의 w/2개의 token을 참고하는 attention.
  - g개의 global token(전체 시퀀스를 참고하는 global tokens)
  - 즉, Q(Query)가 K(Key)의 모든 토큰을 참고하지 않고, 위 3가지 방법을 통해 참고할 Key를 선택하여 연산 효율성을 높인다.

- Big Bird의 Generalized Attention이 널리 사용되지 않는 이유
  - Big Bird는 긴 시퀀스를 효율적으로 처리하기 위해 희소한(sparse) 어텐션 메커니즘을 도입하여 로컬 어텐션, 랜덤 어텐션, 글로벌 어텐션을 결합하여 시퀀스 길이에 선형적으로 스케일링되는 효율성을 제공하지만 다음과 같은 이유로 잘사용되지 않음
  
  - 복잡한 구현과 호환성 문제: Big Bird의 어텐션 메커니즘은 기존 Transformer 아키텍처와 비교하여 구현이 복잡함.
  - 하드웨어 최적화 부족: 희소 어텐션은 이론적으로는 효율적이지만, 실제 하드웨어(GPU 등)에서의 최적화가 어려움.
  - 성능 이점의 제한: 일부 작업에서는 Big Bird의 성능 향상이 기존 모델 대비 크게 두드러지지 않음.

- Flash Attention

  - Flash Attention은
    - 기존 Self-Attention 메커니즘은 Query, Key, Value 행렬을 사용하여 모든 토큰 간의 상호작용을 계산하므로 이 과정에서 모든 토큰 간의 상호작용을 저장한다. 그래서 메모리 사용량이 𝑂(𝑛2)로 매우 커진다.
    - GPU 메모리 한계로 인해 긴 시퀀스를 처리하기 어렵고, 많은 메모리 접근(I/O)이 필요하여 병목 현상이 발생한다.

  - Flash Attention의 메모리 최적화
    - Flash Attention은 메모리 사용량을 줄이기 위해 계산과 저장 방식을 재구성한다.
      - 기존 어텐션에서는 소프트맥스 확률, 가중치, 중간 행렬을 모두 메모리에 저장하는데, Flash Attention은 이를 즉시 사용하고 삭제하는 방식으로 메모리 사용을 최소화한다.
    - 계산 순서를 변경하여 소프트맥스, 어텐션 가중치, 출력 계산을 동시에 수행한다. 이렇게 하면 중간 값을 메모리에 저장하지 않고도 정확한 어텐션 값을 계산할 수 있다.

  - I/O 최적화와 GPU 연산 효율성
    - GPU에서의 메모리 대역폭은 매우 중요한데 Flash Attention은 메모리 접근을 최소화하고 연속적으로 데이터를 처리하여 I/O 병목을 줄이고 GPU의 연산 유닛을 최대한 활용한다.
    - 기존 방식은 Query, Key, Value 행렬을 반복적으로 GPU 메모리에서 불러와야 했으나, Flash Attention은 필요한 데이터만 빠르게 접근하여 처리할 수 있도록 최적화한다.

  - I/O-awareness
    - Flash Attention은 GPU의 shared memory와 global memory 접근 패턴을 최적화하여 데이터 전송을 줄이고, GPU 내에서 데이터의 입출력 효율성을 높인다.
    - 이를 통해 I/O 대기 시간을 줄이고, GPU의 계산 리소스를 더 많이 활용할 수 있다.

  - 정리
    - 메모리 및 계산 효율성: Flash Attention은 어텐션 메커니즘의 메모리 사용량과 계산 복잡도를 줄여줌.
    - 정확한 어텐션 값 계산: 근사치가 아닌 정확한 어텐션 값을 계산하면서도 효율성을 유지. 이는 모델의 성능을 저하시키지 않으면서도 속도를 향상.
    - 쉬운 통합 및 호환성: 기존의 Transformer 아키텍처에 큰 변경 없이도 통합할 수 있어, 쉽게 적용 가능.



- BERT의 경우 BertForMaskedLM 사용
- 모델의 cls 부분은 MLM 작업을 위한 예측 헤드로 모델의 마지막 출력 레이어를 사용하여 각 단어의 확률을 계산

In [1]:
!pip install transformers
!pip install sentencepiece

from transformers import BigBirdTokenizer, BigBirdForMaskedLM
import torch

# 모델과 토크나이저 불러오기
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')
model = BigBirdForMaskedLM.from_pretrained('google/bigbird-roberta-base')
model

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/846k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

BigBirdForMaskedLM(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(50358, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

#67

In [2]:
inputs = ["I like reading [MASK].", "I like driving a [MASK].","The world is facing with a [MASK] [MASK] crisis. We are all suffering from infectious diseases."]
answers = ["I like reading book.", "I like driving a car.", "The world is facing with a pandemic crisis. We are all suffering from infectious diseases."]

#68

In [5]:
# 리스트 타입의 빈 컨테이너 변수 준비
encoded_inputs = []
encoded_labels =  []

# inputs와 answers를 zip을 통해 묶은 후
# inputs와 answers의 입력 문장 하나씩 각기 i와 l 변수로 불러내어 반복 루프 실행
for i, l in zip(inputs, answers):
  # i와 l변수를 각기 토크나이저로 인코딩 후 컨테이너 변수에 각기 순서대로 저장
  # 특히 아래 두번째 코드는 l변수의 토크나이저 인코딩 후 input_ids만 저장
  encoded_inputs.append(tokenizer(i, return_tensors="pt"))
  encoded_labels.append(tokenizer(l, return_tensors="pt")["input_ids"])

# 69

In [4]:
# encoded_inputs와 encoded_labels를 zip()을 통해 묶고
# 그 둘의 요소를 각기 input과 label이라는 변수로 불러내어 반복 반복 루프 실행
for input, label in zip(encoded_inputs, encoded_labels):
  # 모델에 input과 label 투입
  # input은 **kwargs 형태로서 키(key)와 키값(value) 모두 전달
  outputs = model(**input, labels=label)
  # 손실 추출
  loss = outputs.loss
  # 로짓 추출
  logits = outputs.logits

  # 손실 출력. 이때 item( )은 텐서 loss의 값을 파이썬 숫자로 추출
  print(f"loss：{loss.item()}")

  # len(logits[0])은 로짓의 첫번째 차원의 개수이며
  # range(1, leg(logits[0]))은 1,2,3,...그리고 로짓의 첫번째 차원의 개수까지 범위
  # logits[0][i]는 로짓의 첫번째 차원의 i번째 요소
  # argmax(-1)은 logits[0][i]의 요소의 마지막 축(axis)에서 최대값을 갖는 인덱스를 추출
  # 해당 인덱스를 기준으로 디코딩 후 그 결과를 순서대로 묶어서 출력
  print(f"prediction：{' '.join([tokenizer.decode(logits[0][i].argmax(-1)) for i in range(1, len(logits[0]))])}")

  # lable[0][1:-1]은 lable의 첫번째 차원의 처음과 끝을 제외한 모든 요소를 의미
  # 이를 디코딩 후 출력
  print(f"answer：{tokenizer.decode(label[0][1:-1])}")
  # 출력시 줄바꾸기
  print('\n')

Attention type 'block_sparse' is not possible if sequence_length: 7 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


loss：11.183555603027344
prediction：i like reading it . i
answer：I like reading book.


loss：8.306910514831543
prediction：its like driving a car . a
answer：I like driving a car.


loss：4.296058177947998
prediction：the world is facing with a global health crisis . we are all suffering from infectious diseases . .
answer：The world is facing with a pandemic crisis. We are all suffering from infectious diseases.




# 70

### Pegasus(Pre-training with Extracted Gap-sentences for Abstractive Summarization)

구글이 ICML2020 컨퍼런스(icml.cc/Conferences/2020)에서 제안한 PEGASUS 모델은 트랜스포머를 사용하고 문장 요약에 특화된 사전학습 모델입니다. BERT의 MLM(Masked Language Model)은 단어를 마스킹하고 나서 빈칸 채우기(fill-in-the-blanks) 문제를 풀어서 모델 성능을 개선한데 반해, PEGASUS의 GSG(Gap Sentence Generation)는 문장 단위로 마스킹하고 이를 예측하는 사전학습을 실행합니다.

- PEGASUS를 사용해서 자동 문장 요약을 실시 실습

[논문](https://arxiv.org/abs/1912.08777)  
[한글 리뷰](https://velog.io/@jaehyeong/Paper-Review-PEGASUSPre-training-with-Extracted-Gap-sentences-for-Abstractive-Summarization)

#### Text Summarization
![](https://velog.velcdn.com/images%2Fjaehyeong%2Fpost%2Fa6e87a99-815a-4570-94d2-7083c0f065ac%2Fextractive-abstractive.PNG)

-  Extractive: 원문 텍스트로부터 주요 Sentence를 원문 그대로 추출해내는 방식.
-  Abstractive: 원문 텍스트를 보고 생각과 느낌을 한 줄 요약하듯이 표현하는 방식.

#### 아키텍처
![](https://velog.velcdn.com/images%2Fjaehyeong%2Fpost%2Fe531875f-1f6f-4c1a-afb4-0e6bf0fcaf10%2FPEGASUS-architecture.PNG)

![](https://velog.velcdn.com/images%2Fjaehyeong%2Fpost%2F64580d60-3b03-4b97-83fd-23db2c4f674d%2Fpegasus-transformer.gif)

- Gap-Sentences-Generation(GSG)
  - PEGASUS는 기본적으로 encoder-decoder기반의 Transformer구조를 가짐.
  - 기존 MLM(Masked Language Model)과 유사하게 Input text의 일부를 masking하여 Encoder의 input으로 보내는데 PEGASUS는 기존 MLM과 다른 점은 바로 Sentence 자체를 masking한다.
  - 기존 MLM 모델들은 token 단위로 masking하여 masked token을 예측하는 방식으로 학습을 진행하지만, PEGASUS는 Input Document로부터 Sentence 단위로 Masking을 한 후 남은 Sentence를 기반으로 masked sentence를 예측하는 방식으로 학습한다.
  - 논문에서는 이러한 방식을 Gap-Sentences-Generation(GSG)라고 한다.

- 핵심 아이디어
  - Summarization을 수행하기 위해서는 input document와 그에 맞는 summary text가 쌍으로 활용되어야 한다. 하지만 단순히 extractive 방식으로 summary를 추출하게 되면 모델은 단순히 sentence를 copy하는 방식으로 학습이 되기 때문에, 저자는 최근 masking words와 contiguous spans의 성공에 영감을 받아 GSG를 수행한다고 설명한다.
  - contiguous spans: 한 문장에서 여러 단어를 연속적으로 선택하여 마스킹하는 기법.

- GSG 수행 방식
  - Select and mask whole sentences form documents.
  - Concatenate the gap-sentences into a pseudo-summary.
  - The corresponding position of each selected gap sentence is replaced by a mask token [MASK1] to inform the model
  - 여기서 gap sentence 비율은 GSR(Gap Sentences Ratio)에 의해 결정되는데 이는 문서의 전체 sentence에서 선택된 gap sentence의 비율을 의미하고, 다른 Masked Language Model에서의 mask rate와 유사한 개념.
  - 해당 논문에서는 GSR의 비율에 따른 성능을 실험하였는데 데이터셋에 따라 성능 편차가 있었지만, 최종적으로 GSR을 30%로 선택.




어떤 문장이 gap sentence로 선택이 되는걸까?

![](https://velog.velcdn.com/images%2Fjaehyeong%2Fpost%2F5cba21b4-1799-4998-aef0-ccb15da4c97c%2Fgap-sentence-select.PNG)

- Random
  - 설명: 문서에서 랜덤하게 m개의 문장을 선택하는 방식.
  - 의미: 선택된 문장이 문서 내에서 중요한 문장인지 여부와 상관없이 무작위로 선택. 학습 데이터의 다양성을 높이기 위한 전략으로, 특정 문장에 편향되지 않고 모델이 전반적인 문장 구조를 학습할 수 있다.
  - 장점: 다양한 패턴의 데이터를 학습할 수 있음.
  - 단점: 중요한 문장이 선택되지 않을 수 있으며, 요약의 품질이 떨어질 수 있음.
- Lead
  - 설명: 문서의 첫 m개의 문장을 선택하는 방식.
  - 의미: 문서의 서두에 중요한 정보가 많이 포함되는 경우가 많기 때문에, 이 전략은 초기 문장이 중요한 정보를 포함하고 있을 것이라는 가정 하에 첫 문장들을 요약문으로 선택.
  - 장점: 뉴스 기사나 보고서처럼 서두에 중요한 정보가 요약된 문서에 적합.
  단점: 문서 구조에 따라 서두에 중요한 정보가 없는 경우 요약 성능이 낮아질 수 있음.
- Principal
  - 설명: selected sentence와 remaining sentence 간의 ROUGE1-F1 score를 기반으로 가장 중요한 m개의 문장을 선택하는 방식.
  - 의미: 이 방식은 문서 내의 각 문장이 나머지 문장들과 얼마나 중요한 정보를 공유하고 있는지를 측정. ROUGE1-F1 점수는 선택된 문장이 나머지 문장들과 비교하여 얼마나 중요한지를 나타내므로 이 점수를 기준으로, 문서의 핵심 내용을 잘 요약할 수 있는 문장을 선택한다.
  - 세부 옵션:
    - Ind/Seq:
      - Ind(Independent): 각 문장을 독립적으로 평가하여 선택.
      - Seq(Sequential): 문장을 순차적으로 선택하여 문맥이나 순서를 고려함.

    - Orig/Uniq:
      - Orig(Original): 문서 내의 원래의 모든 문장을 사용하여 평가.
      - Uniq(Unique): 반복되는 문장이나 중복되는 정보를 제거하여 평가.
  - 장점: 문서 내에서 가장 중요한 정보를 포함하는 문장을 선택할 수 있음.
  - 단점: 계산 비용이 크고, 특정 문서 구조에 따라 선택이 부정확할 수 있음.

- 각 방식의 역할과 차이점
  - Random
    - 다양성 확보.
    - 특정 문장이 아닌 다양한 패턴의 학습.
  - Lead
    - 서두에 중요한 정보가 있는 경우 효과적.
    - 중요한 정보를 담고 있는 초기 문장 선택.
  - Principal
    - 문서 전체의 요약 성능을 높일 수 있는 문장 선택.
    - ROUGE1-F1 점수를 기반으로 문서의 핵심 문장 추출.

![](https://velog.velcdn.com/images%2Fjaehyeong%2Fpost%2F6daf8d28-f3e7-4640-b4c3-d24263666469%2Fgap-sentence-select-example.PNG)

- Masked Language Model(MLM)
  - BERT에서는 input text의 15%의 token을 선택하여, 그 중 80%는 mask token으로 변환하고, 10%는 random token, 나머지 10%는 그대로 사용했다.
  - PEGASUS는 GSG와 MLM이 동시에 적용했지만 MLM이 downstream task의 성능 향상에 영향을 주지 않아 최종 모델에서는 MLM을 포함하지 않았다.


- 결과
![](https://velog.velcdn.com/images%2Fjaehyeong%2Fpost%2F80a46d99-bcbd-4f43-b04b-c61fac052779%2FPEGASUS-result2.PNG)
![](https://velog.velcdn.com/images%2Fjaehyeong%2Fpost%2Fed1e0f0b-3d89-4243-91c9-79d02d17e905%2FPEGASUS-result1.PNG)

  - PEGASUS모델은 이전 SOTA모델 대비 모든 12개의 downstream tasks에서 모두 SOTA를 달성.
  - PAGASUS-large 모델을 2000 steps, 256 batch-size, 0.0005 learning-rate로 fine-tuning하였을 때, 100개의 examples만으로도 기존 20k~200k개로 학습된 Transformer-base 모델과 유사한 성능을 달성.
  - 1000개의 examples를 사용하였을때 12개 데이터 셋중 6개의 데이터 셋에서 SOTA를 달성.
  - 기존 모델 대비 적은 비용으로 높은 성능을 달성.


#### 결론
- Abstractive summmarization이라는 특정 task를 위해 GSG(Gap-Sentence Generation) pre-training 기법 적용.
- GSG에서 principal sentence selection을 위해 다양한 방법을 적용.
- 적은 리소스 비용(ex, 1000 examples)만으로도 대부분의 결과에서 SOTA 달성.


In [6]:
# 런타임 1분 소요

# 코랩이 메모리가 갱신(clear)된 경우만 아래 두 줄 코딩에서
# 주석 기호(#)를 제거하고 실행
#!pip install transformers
#!pip install sentencepiece

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

# 모델명
model_name = 'google/pegasus-xsum'

# 가능한 경우 GPU를 사용하고, 그렇지 않은 경우 CPU를 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델 및 토크나이저 불러오기
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

# 71

In [9]:
# 여러 문장을 한꺼번에 인용할 때는 인용부호로 """를 사용
inputs = [
          """
          Pretraining large neural language models, such as BERT, has led to impressive gains on many natural language processing (NLP) tasks.
          However, most pretraining efforts focus on general domain corpora, such as newswire and Web. A prevailing assumption is that even domain-specific pretraining can benefit by starting from general-domain language models.
          Recent work shows that for domains with abundant unlabeled text, such as biomedicine, pretraining language models from scratch results in substantial gains over continual pretraining of general-domain language models.
          """
]

# inputs에 담긴 문장을 토크나이저로 인코딩
batch = tokenizer(inputs, truncation=True, padding='longest', return_tensors="pt").to(device)

#72

In [10]:
# 입력문장의 인코딩 결과가 담긴 batch 변수를 **kwargs 형식으로 투입
# model.generate()가 요약 문장을 생성하고 결과는 인코딩되어 있음
translated = model.generate(**batch)

# 위의 인코딩된 요약 문장을 디코딩
# 디코딩 시 [CLS] 등의 특수 토큰은 무시
generated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

# 디코딩 텐서의 첫번째 요소(?차원 추후 재확인 필요) 출력
print(generated_text[0])

Pretraining large neural language models can lead to substantial gains over continual pretraining of general-domain language models.
